In [20]:
import requests
import json
from pprint import pprint
from dotenv import load_dotenv
import os
from geopy.distance import geodesic
import json
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:

def getPlaces(latitude, longitude, api_key):
    url = "https://places.googleapis.com/v1/places:searchNearby"

    data = {
        "maxResultCount": 20,
        "rankPreference": "DISTANCE",
        "locationRestriction": {
            "circle": {
                "center": {
                    "latitude": latitude,
                    "longitude": longitude
                },
                "radius": 50000.0
            }
        }
    }

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.displayName,places.location,places.formattedAddress,places.types,places.primaryType"
    }

    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()  # Will raise an HTTPError for non-200 responses

    return response.json()

In [17]:

# Define the endpoint and payload
url = "https://places.googleapis.com/v1/places:searchNearby"

data = {
    "maxResultCount": 20,
    "rankPreference": "DISTANCE",
    "locationRestriction": {
        "circle": {
            "center": {
                "latitude": 34.861743,
                "longitude": -82.40443
            },
            "radius": 50000.0
        }
    }
}

# Headers to include specific fields
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": API_KEY,
    "X-Goog-FieldMask": "places.displayName,places.location,places.formattedAddress,places.types,places.primaryType"
}

# Send the request
response = requests.post(url, json=data, headers=headers)
places_data = response.json()

print(response)



<Response [200]>


In [8]:
pprint(places_data)

{'places': [{'displayName': {'languageCode': 'en',
                             'text': 'Rutherford St & W Stone Ave'},
             'formattedAddress': 'Greenville, SC 29609, USA',
             'location': {'latitude': 34.861665599999995,
                          'longitude': -82.4044155},
             'primaryType': 'bus_stop',
             'types': ['bus_stop',
                       'transit_station',
                       'point_of_interest',
                       'establishment']},
            {'displayName': {'languageCode': 'en',
                             'text': 'Triune Mercy Center'},
             'formattedAddress': '222 Rutherford St, Greenville, SC 29609, USA',
             'location': {'latitude': 34.8620266, 'longitude': -82.4043009},
             'primaryType': 'church',
             'types': ['church',
                       'place_of_worship',
                       'point_of_interest',
                       'establishment']},
            {'displayName': {'lang

In [11]:
#print all values
for location in places_data.get("places", []):
    address = location.get('formattedAddress', 'NULL')
    lat = location.get('location', {}).get('latitude', 'NULL')
    lon = location.get('location', {}).get('longitude', 'NULL')
    primary_type = location.get('primaryType', 'NULL')
    display_name = location.get('displayName', {}).get('text', 'NULL')

    print(f"{address}\n{lat}\n{lon}\n{primary_type}\n{display_name}\n")


Greenville, SC 29609, USA
34.861665599999995
-82.4044155
bus_stop
Rutherford St & W Stone Ave

222 Rutherford St, Greenville, SC 29609, USA
34.8620266
-82.4043009
church
Triune Mercy Center

222 Rutherford St, Greenville, SC 29609, USA
34.8620136
-82.4041982
food
Triune Mercy Center - Food Distribution Center

214 Rutherford St, Greenville, SC 29609, USA
34.8614047
-82.4040622
barbecue_restaurant
Lewis Barbecue Greenville

Greenville, SC 29609, USA
34.861143999999996
-82.404657
bus_stop
Rutherford St & Echols St

413 W Stone Ave Suite 101, Greenville, SC 29609, USA
34.8621931
-82.405002
veterinary_care
Stone Ave Vet

301 Rutherford St suite e, Greenville, SC 29609, USA
34.8621972
-82.4052147
insurance_agency
Fleming Insurance

201 Rutherford St, Greenville, SC 29609, USA
34.861001099999996
-82.40499249999999
fast_food_restaurant
Hardee’s

204 Rutherford St, Greenville, SC 29609, USA
34.86082
-82.40419
fast_food_restaurant
Arby's

307 Rutherford St, Greenville, SC 29609, USA
34.86272539

In [43]:
#get website
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 2)
for location in places_data.get("places", []):
    # Open the target URL
    driver.get("https://www.google.com/maps/place/")
    
    businessAddress=location.get('formattedAddress', 'NULL')
    print(businessAddress)
    # Clear any pre-filled text and enter your query
    if not businessAddress[0].isdigit():
        print("not an address")
        continue
        

    search_input = wait.until(EC.visibility_of_element_located((By.ID, "searchboxinput")))

    search_input.send_keys(businessAddress)
    
    search_input.send_keys(Keys.ENTER)

    businessName=location.get('displayName', {}).get('text', 'NULL')
    xpath_selector = f"//button[@aria-label='{businessName}']"
    
    
    # Wait for button to be clickable
    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_selector)))
    except:
        continue
    
    # Click the button
    button.click()
    
    
    # Locate the element using the 'data-tooltip' attribute
    try:
        element = wait.until(
            EC.element_to_be_clickable((
                By.XPATH,
                "//a[contains(@class, 'CsEnBe') and @data-tooltip='Open website']"
            ))
        )
    except:
        print("no website")
        continue
    
    
    # Get the href attribute
    href_value = element.get_attribute('href')
    
    # Print the href value
    print("Website URL:", href_value)
    # Close the driver
    #driver.quit()


Greenville, SC 29609, USA
not an address
222 Rutherford St, Greenville, SC 29609, USA
Website URL: http://triunemercy.org/
222 Rutherford St, Greenville, SC 29609, USA
Website URL: https://www.triunemercy.org/food.html
214 Rutherford St, Greenville, SC 29609, USA
Website URL: https://www.lewisbarbecue.com/
Greenville, SC 29609, USA
not an address
413 W Stone Ave Suite 101, Greenville, SC 29609, USA
Website URL: http://www.stoneavevet.com/
301 Rutherford St suite e, Greenville, SC 29609, USA
201 Rutherford St, Greenville, SC 29609, USA
Website URL: https://locations.hardees.com/sc/greenville/201-rutherford-street
204 Rutherford St, Greenville, SC 29609, USA
307 Rutherford St, Greenville, SC 29609, USA
Website URL: http://www.caglecenter.com/
329 W Stone Ave, Greenville, SC 29609, USA
Website URL: https://www.giraffeweb.net/
413 W Stone Ave #102, Greenville, SC 29609, USA
Website URL: https://www.carolinaanimaleye.com/
332 W Stone Ave, Greenville, SC 29609, USA
323 W Stone Ave, Greenvill

In [26]:
print(places_data.get("places", [])[1].get('displayName', {}).get('text', 'NULL'))
print(places_data.get("places", [])[2].get('displayName', {}).get('text', 'NULL'))

Triune Mercy Center
Triune Mercy Center - Food Distribution Center


In [22]:
#get furthest point (radius of area)
reference_point = (34.861743, -82.40443)

# Assume places_data is already defined from your Places API v1 response
location_array = []
for i in places_data['places']:
    lat = i['location']['latitude']
    lng = i['location']['longitude']
    location_array.append((lat, lng))

# Calculate distances from reference point
distances = [
    (geodesic(reference_point, loc).meters, loc)
    for loc in location_array
]

# Find the furthest point
max_distance, furthest_point = max(distances, key=lambda x: x[0])

print(f"Furthest point: {furthest_point}")
print(f"Distance: {max_distance:} meters")

Furthest point: (34.860597299999995, -82.405853)
Distance: 181.8972281172954 meters


In [ ]:
def collect_places_data(current_circle_number):
    #get the current radius of the circle
    current_area=calculate_geojson_area(rf"C:\Users\Joe\Desktop\geojsons\circle{current_circle_number}.geojson")
    #get the calculated maximum area of the circle
    with open(r"C:\Users\Joe\Desktop\geojsons\geojson_total_areas.json") as f:
        areas = json.load(f)
    max_area=areas[f"circle{current_circle_number}.geojson"]
    
    while current_radius